In [1]:
import requests
import folium
import geopandas as gpd
from shapely.geometry import shape
from pyproj import CRS
import json
import os

Studi Kasus 1: Penentuan Fleet/Mobil Terdekat & Terbaik Berbasis Spasial dan Multi-Kriteria

In [2]:
# Definisikan URL
mobil_fleet = 'https://geoserver.mapid.io/layers_new/get_layer?api_key=70b6758f25e94e6a849a32d935bce369&layer_id=69885cbe2d33abfe640dff7e&project_id=697e526a949d92a51f0b4816'
pelanggan = 'https://geoserver.mapid.io/layers_new/get_layer?api_key=70b6758f25e94e6a849a32d935bce369&layer_id=69885de72d33abfe640e60c8&project_id=697e526a949d92a51f0b4816'
area = 'https://geoserver.mapid.io/layers_new/get_layer?api_key=70b6758f25e94e6a849a32d935bce369&layer_id=69885e69bbaa47f3ce175cde&project_id=697e526a949d92a51f0b4816'

# 1. Ambil data (Gunakan nama variabel yang konsisten)
requests_mobil = requests.get(mobil_fleet)
requests_pelanggan = requests.get(pelanggan)
requests_area = requests.get(area)

# 2. Cek status (Pastikan menggunakan nama variabel yang didefinisikan di atas)
if requests_mobil.status_code == 200 and requests_pelanggan.status_code == 200 and requests_area.status_code == 200:
    data_mobil = requests_mobil.json()
    data_pelanggan = requests_pelanggan.json()
    data_area = requests_area.json()

    # 3. Buat GeoDataFrame
    gdf_mobil = gpd.GeoDataFrame.from_features(data_mobil['features'])
    gdf_pelanggan = gpd.GeoDataFrame.from_features(data_pelanggan['features'])
    gdf_area = gpd.GeoDataFrame.from_features(data_area['features'])

    # 4. Atur CRS
    gdf_mobil.crs = CRS.from_epsg(4326)
    gdf_pelanggan.crs = CRS.from_epsg(4326)
    gdf_area.crs = CRS.from_epsg(4326)
    
    # Pastikan baris print ini menjorok ke dalam (sejajar dengan gdf_area.crs)
    print("Data berhasil dimuat!")
else:
    print("Gagal mengambil data dari API.")

Data berhasil dimuat!


Filter Mobil Aktif & Kapasitas Cukup


In [3]:
def filter_mobil_aktif(df, target_penumpang):
    mask = (df['STATUS'] == 'AKTIF') & (df['KAPASITAS'] >= jumlah_penumpang)
    return df[mask].copy()

jumlah_penumpang = gdf_pelanggan.iloc[0]['JUMLAH_PENUMPANG']
gdf_mobil_aktif = filter_mobil_aktif(gdf_mobil, jumlah_penumpang)

Hitung Jarak Mobil ke Pelanggan

In [4]:
def hitung_jarak(gdf_mobil_aktif, pelanggan_point):
    gdf_mobil_aktif['jarak'] = gdf_mobil_aktif['geometry'].distance(pelanggan_point)
    return gdf_mobil_aktif

pelanggan_point = gdf_pelanggan.iloc[0]
gdf_mobil_aktif = hitung_jarak(gdf_mobil_aktif, pelanggan_point['geometry'])

C:\Users\ojiar\AppData\Local\Temp\ipykernel_22024\3199529981.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_mobil_aktif['jarak'] = gdf_mobil_aktif['geometry'].distance(pelanggan_point)


Spatial Join: Dapatkan Info Area Mobil & Pelanggan


In [5]:
def spatial_join(gdf_obj, gdf_area):
    return gpd.sjoin(gdf_obj, gdf_area, how='left', predicate='within')

mobil_area = spatial_join(gdf_mobil_aktif, gdf_area)
pelanggan_area = spatial_join(gdf_pelanggan, gdf_area)

Skoring Multi-Kriteria Mobil

In [6]:
# 2. FUNGSI PENDUKUNG (Scoring & Jarak)
def hitung_jarak_km(lat1, lon1, lat2, lon2):
    R = 6371  # Radius bumi dalam km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c

def skoring_mobil(row, pelanggan_row):
    skor = 0
    # Jarak (semakin dekat semakin tinggi)
    jarak_km = hitung_jarak_km(row.geometry.y, row.geometry.x, pelanggan_row.geometry.y, pelanggan_row.geometry.x)
    
    if jarak_km < 1:
        skor += 100
    elif jarak_km < 2:
        skor += 50
    elif jarak_km < 3:
        skor += 25
    
    # Kondisi Lingkungan
    skor += 10 if str(row.get("HUJAN", "")).upper() == 'TIDAK' else 0
    skor += 10 if str(row.get("BANJIR", "")).upper() == 'TIDAK' else 0
    
    # Kepadatan & Traffic
    skor += 10 if row.get('KEPADATAN') == 'RENDAH' else 5 if row.get('KEPADATAN') == 'SEDANG' else 0
    skor += 10 if row.get('TRAFFIC') == 'LANCAR' else 5 if row.get('TRAFFIC') == 'SEDANG' else 0
    
    # Rating Driver
    skor += 10 if row.get('RATING', 0) >= 4.8 else 5
    return skor

 Pilih Mobil Terbaik Berdasarkan Skor dan Jarak

In [9]:
def pilih_mobil_terbaik(mobil_area, pelanggan):

    # Hitung skor untuk setiap mobil
    mobil_area['SKOR'] = mobil_area.apply(lambda row: skoring_mobil(row, pelanggan), axis=1)
   
    # Hitung jarak dalam km
    mobil_area['JARAK_KM'] = mobil_area.apply(lambda row: hitung_jarak_km(
        row.geometry.y, row.geometry.x, pelanggan.geometry.y, pelanggan.geometry.x), axis=1)
    
    # Urutkan berdasarkan JARAK_KM (asc), lalu SKOR (desc)
    mobil_area_sorted = mobil_area.sort_values(['JARAK_KM', 'SKOR'], ascending=[True, False])
    return mobil_area_sorted

pelanggan= gdf_pelanggan.iloc[0]
mobil_area_sorted = pilih_mobil_terbaik(mobil_area, pelanggan)

# tampilkan mobil terpilih
mobil_terpilih = mobil_area_sorted.iloc[0]
print("Mobil Terpilih:")
print(mobil_terpilih[['NAMA_DRIVER', 'TIPE', 'KAPASITAS', 'RATING', 'JARAK_KM', 'SKOR']])

# MOBIL TIDAK TERPILIH
MOBIL_TIDAK_TERPILIH = mobil_area_sorted.iloc[1:]  

# MENGABAIKAN BARIS PERTAMA TERPILIH
print("\nMobil Tidak Terpilih (5 Teratas):")
print(MOBIL_TIDAK_TERPILIH[['NAMA_DRIVER', 'TIPE', 'KAPASITAS', 'RATING', 'JARAK_KM', 'SKOR']])



NameError: name 'math' is not defined

In [32]:
import json

mobil_terpilih_geojson = {
    "type": "Feature",
    "properties": {
        "NAMA_DRIVER": mobil_terpilih['NAMA_DRIVER'],
        "TIPE": mobil_terpilih['TIPE'],
        "KAPASITAS": int(mobil_terpilih['KAPASITAS']),
        "RATING": float(mobil_terpilih['RATING']),
        "JARAK_KM": float(mobil_terpilih['JARAK_KM']),
        "SKOR": int(mobil_terpilih['SKOR'])
        },
        "geometry": {
            "type": "Point",
            "coordinates": [mobil_terpilih.geometry.x, mobil_terpilih.geometry.y]}}

# Simpan ke folder tujuan
output_dir = r"E:\MAPID 2026\Task 7 Analisis Spasial Dalem"
output_path = os.path.join(output_dir, "mobil_terpilih.geojson")
    
with open(output_path, 'w') as f:
    json.dump(mobil_terpilih_geojson, f)